# Raster reconstruct with 0-360 data

In [ ]:
import pygplates
import gplately
import pygmt
import numpy as np
import xarray as xr
import os
import plate_model_manager

In [ ]:
def gplately_raster_to_xarray(raster, raster_name, raster_units):
    
    # check the order of lats
    if raster.lats.min () < raster.lats.max():
        ds_raster = xr.Dataset(data_vars=dict(z=(["lat", "lon"], 
                                                 raster.data, {"long_name": raster_name,
                                                               "units": raster_units,
                                                               "actual_range": np.array([np.nanmin(raster.data), np.nanmax(raster.data)], dtype=np.float32)})),
                               coords=dict(lon=(["lon"], raster.lons.astype('float32'),
                                                {"long_name": "longitude", "units": "degrees_east", "standard_name": "longitude", 
                                                 "actual_range": np.array([raster.lons.min(), raster.lons.max()], dtype=np.float32)}),
                                           lat=(["lat"], (raster.lats).astype('float32'),
                                                {"long_name": "latitude", "units": "degrees_north", "standard_name": "latitude", 
                                                 "actual_range": np.array([raster.lats.min(), raster.lats.max()], dtype=np.float32)})))
    else:
        # otherwise the lats are in DESCENDING order, which means we need to flip the data and lats (otherwise the grid will be upside down later)
        ds_raster = xr.Dataset(data_vars=dict(z=(["lat", "lon"],
                                                 np.flipud(raster.data), {"long_name": raster_name,
                                                                          "units": raster_units,
                                                                          "actual_range": np.array([np.nanmin(raster.data), np.nanmax(raster.data)], dtype=np.float32)})),
                               coords=dict(lon=(["lon"], raster.lons.astype('float32'),
                                                {"long_name": "longitude", "units": "degrees_east", "standard_name": "longitude", 
                                                 "actual_range": np.array([raster.lons.min(), raster.lons.max()], dtype=np.float32)}),
                                           lat=(["lat"], np.flipud(raster.lats).astype('float32'),
                                                {"long_name": "latitude", "units": "degrees_north", "standard_name": "latitude", 
                                                 "actual_range": np.array([raster.lats.min(), raster.lats.max()], dtype=np.float32)})))
        
    return ds_raster

In [ ]:
plate_models = 'plate_models'
if not os.path.exists(plate_models):
    os.makedirs(plate_models)

# import target plate model: Zahirovic et al. (2022).
try:
    S12_model_pmm = plate_model_manager.PlateModelManager().get_model("Seton2012", data_dir=plate_models)
except:
    # if unable to connect to the servers, try to use the local files
    S12_model_pmm = plate_model_manager.PlateModel(model_name="Seton2012", data_dir=plate_models, readonly=True)

# get the plate model components
S12_rotation_filename = S12_model_pmm.get_rotation_model()
S12_static_polygons_filename = S12_model_pmm.get_static_polygons()
S12_coastline_filename = S12_model_pmm.get_coastlines()
S12_topology_features = S12_model_pmm.get_topologies()

# create a GPlately 'plate reconstruction' object
S12_model = gplately.PlateReconstruction(S12_rotation_filename, S12_topology_features, S12_static_polygons_filename, anchor_plate_id=0)

In [ ]:
seton_2012_age_grid = gplately.Raster(
    data=S12_model_pmm.get_raster("AgeGrids", 50),
    plate_reconstruction=S12_model)

In [ ]:
# check longitude extent
seton_2012_age_grid.lons

In [ ]:
# convert to dataset
ds_seton_2012_age_grid = gplately_raster_to_xarray(seton_2012_age_grid, 'age', 'myr')

In [ ]:
ds_seton_2012_age_grid.z.plot()

In [ ]:
# reformat longitudes into 0-360 format
ds_seton_2012_age_grid['lon'] = (ds_seton_2012_age_grid.coords['lon']) % 360
ds_seton_2012_age_grid = ds_seton_2012_age_grid.sortby(ds_seton_2012_age_grid.lon)
ds_seton_2012_age_grid.z.plot()

In [ ]:
# create a gplately plotting object with our plate model
gplot = gplately.PlotTopologies(S12_model, coastlines=S12_coastline_filename,
                                continents=S12_static_polygons_filename,
                                COBs=None, time=0)
gplot.time = 50

# ---- actually plot
region = 'd'
width = 8
projection = 'N180/%sc' % width

pen = '1p'
alpha = 25

label_font = '10p,Helvetica,black'
label_offset = 'j0/0c'
label_position = 'TL'

fig = pygmt.Figure()

pygmt.makecpt(cmap="magma", reverse=True, series=[0, 150, 1])

pygmt.config(FONT_ANNOT=6.5, FONT_LABEL=8, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="3p")
fig.basemap(projection=projection, region=region, frame="lrtb")
fig.grdimage(grid=ds_seton_2012_age_grid.z)
fig.plot(gplot.get_continents(), pen='0.05p,black', region=region, projection=projection)
fig.text(text='%s Ma' % 50, position=label_position, no_clip=True, font=label_font, offset=label_offset)
fig.colorbar(frame=["x+lAge (myr)"], position="JRM+o0.5c/0c+w%sc/0.2c+ef" % (width - 4))
fig.show(width=1000)

Reconstruct to present day

In [ ]:
# add grid as is - do not specify extent. Would hope GPlately would read the extent from the data itself (since it's a netcdf)
raster_50 = gplately.Raster(data=ds_seton_2012_age_grid.z, time=50)
raster_50.plate_reconstruction = S12_model

raster_0 = raster_50.reconstruct(0)  # reconstruct to present-day

ds_raster_0 = gplately_raster_to_xarray(raster_0, 'age', 'myr')

In [ ]:
gplot = gplately.PlotTopologies(S12_model, coastlines=S12_coastline_filename,
                                continents=S12_static_polygons_filename,
                                COBs=None, time=0)
gplot.time = 0

# ---- actually plot
region = 'd'
width = 8
projection = 'N180/%sc' % width
label_font = '10p,Helvetica,black'
label_offset = 'j0/0c'
label_position = 'TL'

fig = pygmt.Figure()

pygmt.makecpt(cmap="magma", reverse=True, series=[0, 150, 1])

pygmt.config(FONT_ANNOT=6.5, FONT_LABEL=8, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="3p")
fig.basemap(projection=projection, region=region, frame="lrtb")
fig.grdimage(grid=ds_raster_0.z)
fig.plot(gplot.get_continents(), pen='0.05p,black', region=region, projection=projection)
fig.text(text='%s Ma' % 0, position=label_position, no_clip=True, font=label_font, offset=label_offset)
fig.colorbar(frame=["x+lAge (myr)"], position="JRM+o0.5c/0c+w%sc/0.2c+ef" % (width - 4))
fig.show(width=1000)

Specifically add global extent in 0-360 format

In [ ]:
raster_50_v2 = gplately.Raster(data=ds_seton_2012_age_grid.z, time=50, extent=[0, 360, -90, 90])
raster_50_v2.plate_reconstruction = S12_model

raster_0_v2 = raster_50_v2.reconstruct(0)  # reconstruct to present-day

ds_raster_0_v2 = gplately_raster_to_xarray(raster_0_v2, 'age', 'myr')

In [ ]:
gplot = gplately.PlotTopologies(S12_model, coastlines=S12_coastline_filename,
                                continents=S12_static_polygons_filename,
                                COBs=None, time=0)
gplot.time = 0

# ---- actually plot
region = 'd'
width = 8
projection = 'N180/%sc' % width
label_font = '10p,Helvetica,black'
label_offset = 'j0/0c'
label_position = 'TL'

fig = pygmt.Figure()

pygmt.makecpt(cmap="magma", reverse=True, series=[0, 150, 1])

pygmt.config(FONT_ANNOT=6.5, FONT_LABEL=8, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="3p")
fig.basemap(projection=projection, region=region, frame="lrtb")
fig.grdimage(grid=ds_raster_0_v2.z)
fig.plot(gplot.get_continents(), pen='0.05p,black', region=region, projection=projection)
fig.text(text='%s Ma' % 0, position=label_position, no_clip=True, font=label_font, offset=label_offset)
fig.colorbar(frame=["x+lAge (myr)"], position="JRM+o0.5c/0c+w%sc/0.2c+ef" % (width - 4))
fig.show(width=1000)

In [ ]:
raster_50_v3 = gplately.Raster(data=ds_seton_2012_age_grid.z, time=50, extent=[0, 360, -90, 90], realign=True)
raster_50_v3.plate_reconstruction = S12_model

raster_0_v3 = raster_50_v3.reconstruct(0)  # reconstruct to present-day

ds_raster_0_v3 = gplately_raster_to_xarray(raster_0_v3, 'age', 'myr')

In [ ]:
gplot = gplately.PlotTopologies(S12_model, coastlines=S12_coastline_filename,
                                continents=S12_static_polygons_filename,
                                COBs=None, time=0)
gplot.time = 0

# ---- actually plot
region = 'd'
width = 8
projection = 'N180/%sc' % width
label_font = '10p,Helvetica,black'
label_offset = 'j0/0c'
label_position = 'TL'

fig = pygmt.Figure()

pygmt.makecpt(cmap="magma", reverse=True, series=[0, 150, 1])

pygmt.config(FONT_ANNOT=6.5, FONT_LABEL=8, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="3p")
fig.basemap(projection=projection, region=region, frame="lrtb")
fig.grdimage(grid=ds_raster_0_v3.z)
fig.plot(gplot.get_continents(), pen='0.05p,black', region=region, projection=projection)
fig.text(text='%s Ma' % 0, position=label_position, no_clip=True, font=label_font, offset=label_offset)
fig.colorbar(frame=["x+lAge (myr)"], position="JRM+o0.5c/0c+w%sc/0.2c+ef" % (width - 4))
fig.show(width=1000)

In [ ]:
raster_50_v4 = gplately.Raster(data=ds_seton_2012_age_grid.z, time=50, realign=True)
raster_50_v4.plate_reconstruction = S12_model

raster_0_v4 = raster_50_v4.reconstruct(0)  # reconstruct to present-day

ds_raster_0_v4 = gplately_raster_to_xarray(raster_0_v4, 'age', 'myr')

In [ ]:
gplot = gplately.PlotTopologies(S12_model, coastlines=S12_coastline_filename,
                                continents=S12_static_polygons_filename,
                                COBs=None, time=0)
gplot.time = 0

# ---- actually plot
region = 'd'
width = 8
projection = 'N180/%sc' % width
label_font = '10p,Helvetica,black'
label_offset = 'j0/0c'
label_position = 'TL'

fig = pygmt.Figure()

pygmt.makecpt(cmap="magma", reverse=True, series=[0, 150, 1])

pygmt.config(FONT_ANNOT=6.5, FONT_LABEL=8, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="3p")
fig.basemap(projection=projection, region=region, frame="lrtb")
fig.grdimage(grid=ds_raster_0_v4.z)
fig.plot(gplot.get_continents(), pen='0.05p,black', region=region, projection=projection)
fig.text(text='%s Ma' % 0, position=label_position, no_clip=True, font=label_font, offset=label_offset)
fig.colorbar(frame=["x+lAge (myr)"], position="JRM+o0.5c/0c+w%sc/0.2c+ef" % (width - 4))
fig.show(width=1000)